# Conjuntos S, E, e R no Algoritmo AOSVR

No algoritmo AOSVR, as amostras de treinamento são particionadas em três conjuntos importantes, cada um com um papel específico no modelo:

## 1. Conjunto S (Suporte Ativo)
- **Descrição**: As amostras no conjunto $S$ são aquelas que mais influenciam diretamente a função de decisão da Support Vector Regression (SVR).
- **Características**: 
  - Os coeficientes $\theta_i$ das amostras em $S$ estão entre $0 < \theta_i < C$, ou seja, não estão no limite superior ou inferior.
  - Estas amostras estão dentro da margem de erro permitida, mas ainda têm influência significativa no modelo.
- **Função no AOSVR**: 
  - Contribuem diretamente para o ajuste do modelo e afetam o valor do bias $b$.
  - São atualizadas no procedimento de **bookkeeping** (contabilidade).

## 2. Conjunto E (Vetores de Suporte no Limite)
- **Descrição**: O conjunto $E$ contém as amostras que estão no limite da margem de erro ou que a violam ligeiramente.
- **Características**: 
  - Os coeficientes dessas amostras são $\theta_i = C$ ou $\theta_i = -C$, ou seja, atingiram os limites permitidos.
  - As amostras em $E$ não podem ser mais ajustadas em direção ao centro da margem de tolerância $\epsilon$.
- **Função no AOSVR**: 
  - Definem os limites da margem e ajudam a determinar as fronteiras de decisão do modelo.
  - Caso o erro preditivo $h(x)$ mude, elas podem ser movidas para o conjunto $S$.

## 3. Conjunto R (Resíduos ou Amostras "Relaxadas")
- **Descrição**: As amostras no conjunto $R$ são aquelas cujo erro $h(x)$ está dentro da margem de tolerância $\epsilon$.
- **Características**: 
  - Essas amostras têm $\theta_i = 0$, pois não influenciam diretamente a função de decisão.
  - Estão dentro da "zona segura" e não precisam de ajuste.
- **Função no AOSVR**: 
  - São ignoradas pelo algoritmo em termos de ajuste do modelo, a menos que o erro delas ultrapasse o limite de tolerância $\epsilon$.
  - Podem ser movidas para os conjuntos $S$ ou $E$ se o erro $h(x)$ violar a condição $|h(x)| > \epsilon$.

## Resumo dos Conjuntos:
- **$S$**: Amostras ativas, com coeficientes $\theta$ entre $0$ e $C$. Contribuem para o ajuste do modelo.
- **$E$**: Amostras nos limites, com coeficientes $\theta = C$ ou $\theta = -C$. São vetores de suporte que definem as fronteiras de decisão.
- **$R$**: Amostras com erro dentro da margem de tolerância $\epsilon$. Possuem $\theta = 0$ e não influenciam o modelo.

Esses conjuntos permitem ao AOSVR focar nas amostras que realmente precisam ser ajustadas, tornando o processo incremental de atualização mais eficiente.



# Onde os Vetores São Atualizados no Algoritmo AOSVR

No algoritmo AOSVR, os coeficientes $ \theta $ (vetores) são atualizados em vários momentos, dependendo do conjunto ao qual a amostra pertence e do progresso na satisfação das condições de Karush-Kuhn-Tucker (KKT).

## 1. Inicialização do Coeficiente $ \theta_c $ (Novo Dado)
- **Quando**: Assim que um novo dado $ (x_c, y_c) $ é incluído no modelo.
- **O que ocorre**: O coeficiente inicial $ \theta_c $ é atribuído como zero ($ \theta_c = 0 $).
- **Por que**: O novo dado ainda não influencia o modelo e precisa ser ajustado até satisfazer as condições KKT.

## 2. Procedimento de Contabilidade (Bookkeeping Procedure)
No procedimento de contabilidade, o coeficiente $ \theta_c $ do novo dado e os coeficientes $ \theta_i $ das amostras nos conjuntos $S$ e $E$ são ajustados conforme as condições do problema.

### Cálculo do $ \theta_c $:
- **Como**: O algoritmo compara os limites calculados para determinar o menor valor que pode ser utilizado para atualizar $ \theta_c $.
- **Equação**:
  $$
  \theta_c = q \cdot \min\left(|L_{c1}|, |L_{c2}|, |L_S|, |L_E|, |L_R|\right)
  $$
  - $ q $ é o sinal determinado pelo valor de $ h(x_c) $, que indica a direção do ajuste.

### Atualização dos $ \theta_i $ para Amostras no Conjunto $S$:
- **Quando**: Se o novo dado $ x_c $ for adicionado ao conjunto $S$, os coeficientes das amostras já presentes em $S$ também podem ser ajustados.
- **Como**: A atualização dos coeficientes segue a **equação (3.6)**, que ajusta os valores de $ \theta_i $ para manter as condições de otimização.
- **Equação (3.6)**:
  $$
  \theta_i^{\text{novo}} = \theta_i^{\text{antigo}} + \Delta \theta
  $$
  - $ \Delta \theta $ depende da interação entre $ x_c $ e as amostras existentes.

## 3. Atualização do Bias $ b $
- **Quando**: O bias $ b $ é atualizado durante o procedimento de contabilidade.
- **Como**: Segue a mesma **equação (3.6)** utilizada para ajustar os coeficientes $ \theta_i $.

## 4. Atualização de $ h(x_i) $ nos Conjuntos $ E $ e $ R $
- **Quando**: Após a atualização dos coeficientes $ \theta $, o erro preditivo $ h(x_i) $ das amostras nos conjuntos $ E $ e $ R $ também é recalculado.
- **Por que**: Se o erro preditivo $ h(x_i) $ violar os limites de tolerância, essas amostras podem ser movidas entre os conjuntos $S$, $E$ e $R$, e seus coeficientes são ajustados conforme necessário.
- **Equação (3.8)**: O erro $ h(x_i) $ é recalculado com base nos novos coeficientes.

## 5. Atualizações de Partições (Conjuntos $S$, $E$, $R$)
Dependendo do **Flag** (indicador que define qual condição foi responsável pela última atualização), as amostras podem ser movidas entre os conjuntos $S$, $E$ e $R$:
- **Flag 1 ou 2**: O novo dado $ x_c $ é movido para o conjunto $S$ ou $E$, e seu coeficiente é atualizado para refletir sua nova posição.
- **Flag 3, 4 ou 5**: Outras amostras podem ser movidas entre os conjuntos $S$ ou $E$, e seus coeficientes são ajustados conforme necessário.

## Resumo:
- Os coeficientes $ \theta $ são atualizados durante o **procedimento de contabilidade**, onde o algoritmo ajusta os valores de $ \theta_c $ (para o novo dado) e $ \theta_i $ (para amostras nos conjuntos $S$ e $E$).
- As atualizações ocorrem com base nos cálculos dos limites $L_{ci}$, e as amostras podem ser movidas entre os conjuntos $S$, $E$ e $R$, dependendo das condições de Karush-Kuhn-Tucker (KKT).
